# Study Planning Agent

# Install Google ADK

- [Documentation Link](https://google.github.io/adk-docs/)

In [ ]:
!pip install google-adk -q

# Configure API Key

In [2]:
import os
import stat
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    # os.environ["GOOGLE_API_KEY"] = 'your_api_key'
except Exception as e:
    print(f"Authentication Error. Details: {e}")

# Agent Setup

In [3]:
# Create the agent directory
AGENT_DIR = "study_agent"
!mkdir -p {AGENT_DIR}
print(f"Agent directory '{AGENT_DIR}' created.")

# Set permissions for the directory
os.chmod(AGENT_DIR, stat.S_IRWXU | stat.S_IRWXG | stat.S_IRWXO)

Agent directory 'study_agent' created.


In [18]:
%%writefile {AGENT_DIR}/agent.py

# This cell writes the main agent.py file.
# It defines all our agents and the custom tools that the main StudyAgent will use.

import json
import os
from typing import List, Dict, Any
import asyncio
import uuid

from google.adk.agents import Agent, LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search, preload_memory
from google.adk.tools.tool_context import ToolContext
from google.genai import types

# --- Configuration ---
# Define a standard retry configuration for all LLM calls (From 1a)
RETRY_CONFIG = types.HttpRetryOptions(
    attempts=5,
    exp_base=2,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)


# --- Agent Definitions ---
# These agents are not exposed directly, but rather used by the orchestrator to generate schedules

def get_planner_agent():
    """Returns an agent for generating study plans."""
    return LlmAgent(
        model=Gemini(model="gemini-2.5-flash-lite", retry_options=RETRY_CONFIG),
        name="PlannerAgent",
        instruction="""You are a curriculum planner. Your task is to generate a JSON-formatted study plan for a given topic.
        The plan should be a list of 3-5 modules, where each module is an object with a 'title' and a 'description'.
        Respond ONLY with the raw JSON list, no other text.
        Example: [{"title": "Module 1: Intro", "description": "What is it?"}, {"title": "Module 2: Core Concepts", "description": "How does it work?"}]"""
    )

def get_teacher_agent():
    """Returns an agent for teaching a single module."""
    return LlmAgent(
        model=Gemini(model="gemini-2.5-flash-lite", retry_options=RETRY_CONFIG),
        name="TeacherAgent",
        instruction="""You are a helpful teacher. You will be given a specific module title to research.
        You MUST use the Google Search tool to find information. After researching, synthesize the information
        into a concise lesson (approx. 200-300 words). Respond ONLY with the lesson content.""",
        tools=[google_search],
    )

def get_quizzer_agent():
    """Returns an agent for generating quizzes."""
    return LlmAgent(
        model=Gemini(model="gemini-2.5-flash-lite", retry_options=RETRY_CONFIG),
        name="QuizzerAgent",
        instruction="""You are a quiz creator. You will be given lesson content. Your job is to create
        a single, 3-question multiple-choice quiz based ONLY on that content. Provide the questions and options.
        Do NOT provide the answers. Respond ONLY with the quiz.""",
    )

def get_evaluator_agent():
    """Returns an agent for evaluating quiz answers."""
    return LlmAgent(
        model=Gemini(model="gemini-2.5-flash-lite", retry_options=RETRY_CONFIG),
        name="EvaluatorAgent",
        instruction="""You are a fair evaluator. You will receive quiz questions and a user's answer.
        Your job is to evaluate the answer, state if it's correct, and provide a brief, helpful explanation.""",
    )

# --- Custom Tools ---
# These tools wrap our specialized agents (Defined above). Uses 2A: Agent as a tool

async def generate_study_plan(topic: str, tool_context: ToolContext) -> str:
    """Generates a new study plan for a topic and saves it to session state. This tool should be called first."""
    try:
        planner_agent = get_planner_agent()
        
        runner = InMemoryRunner(agent=planner_agent, app_name=planner_agent.name)
        
        user_id = tool_context.session.user_id
        session_id = f"planner_session_{uuid.uuid4().hex}"
        
        # Get the runner's session service and create the session
        sub_session_service = runner.session_service
        await sub_session_service.create_session(
            app_name=planner_agent.name,
            user_id=user_id, 
            session_id=session_id
        )
        
        # Convert the topic string into a Content object
        query_content = types.Content(role="user", parts=[types.Part(text=topic)])
        
        response_text = ""
        async for event in runner.run_async(
            new_message=query_content,
            user_id=user_id,
            session_id=session_id
        ):
            if event.is_final_response() and event.content:
                response_text = event.content.parts[0].text
                break
        
        # Validate and save to session state (3a)
        try:
            plan_data = json.loads(response_text)
            tool_context.state["study_plan"] = json.dumps(plan_data)
            tool_context.state["current_module_index"] = 0
            tool_context.state["current_topic"] = topic
            return f"Study plan created for {topic}:\n{response_text}"
        except json.JSONDecodeError:
            return "Error: The planner agent did not return valid JSON. Please try again."
    except Exception as e:
        return f"Error generating study plan: {e}"

async def teach_next_module(tool_context: ToolContext) -> str:
    """Teaches the next module from the plan stored in session state. Requires generate_study_plan to be called first."""
    plan_str = tool_context.state.get("study_plan")
    index = tool_context.state.get("current_module_index", 0)
    
    if not plan_str:
        return "Error: No study plan found. Please call generate_study_plan first."
    
    try:
        plan = json.loads(plan_str)
        if index >= len(plan):
            return "Congratulations! You have completed all modules in the plan."
        
        module = plan[index]
        module_title = module.get("title", "No Title")
        
        teacher_agent = get_teacher_agent()
        
        runner = InMemoryRunner(agent=teacher_agent, app_name=teacher_agent.name)
        
        user_id = tool_context.session.user_id
        session_id = f"teacher_session_{uuid.uuid4().hex}"

        sub_session_service = runner.session_service
        await sub_session_service.create_session(
            app_name=teacher_agent.name, 
            user_id=user_id, 
            session_id=session_id
        )
        
        query_content = types.Content(role="user", parts=[types.Part(text=module_title)])
        
        lesson_content = ""
        async for event in runner.run_async(
            new_message=query_content,
            user_id=user_id,
            session_id=session_id
        ):
            if event.is_final_response() and event.content:
                lesson_content = event.content.parts[0].text
                break
        
        # Save lesson and update index in state
        tool_context.state["current_lesson"] = lesson_content
        tool_context.state["current_module_index"] = index + 1
        
        return f"--- Module {index + 1}: {module_title} ---\n{lesson_content}"
    except Exception as e:
        return f"Error teaching module: {e}"

async def generate_quiz(tool_context: ToolContext) -> str:
    """Generates a quiz for the most recently taught lesson. Requires teach_next_module to be called first."""
    lesson = tool_context.state.get("current_lesson")
    
    if not lesson:
        return "Error: No lesson has been taught yet. Please complete a module first."
    
    quizzer_agent = get_quizzer_agent()
    
    runner = InMemoryRunner(agent=quizzer_agent, app_name=quizzer_agent.name)
    
    user_id = tool_context.session.user_id
    session_id = f"quizzer_session_{uuid.uuid4().hex}"
    
    sub_session_service = runner.session_service
    await sub_session_service.create_session(
        app_name=quizzer_agent.name, 
        user_id=user_id, 
        session_id=session_id
    )

    query_content = types.Content(role="user", parts=[types.Part(text=lesson)])

    quiz_content = ""
    async for event in runner.run_async(
        new_message=query_content,
        user_id=user_id,
        session_id=session_id
    ):
        if event.is_final_response() and event.content:
            quiz_content = event.content.parts[0].text
            break
    
    tool_context.state["current_quiz"] = quiz_content
    return f"Here is your quiz:\n{quiz_content}\n\nPlease provide your answers in the next message."

async def evaluate_answer(user_answer: str, tool_context: ToolContext) -> str:
    """Evaluates the user's answer against the quiz in session state. Requires generate_quiz to be called first."""
    quiz = tool_context.state.get("current_quiz")
    
    if not quiz:
        return "Error: No quiz has been generated. Please request a quiz first."
    
    eval_prompt = f"Here is the quiz:\n{quiz}\n\nHere is the user's answer:\n{user_answer}\n\nEvaluate the user's answer, state if it's correct, and provide a brief explanation."
    
    eval_agent = get_evaluator_agent()
    
    runner = InMemoryRunner(agent=eval_agent, app_name=eval_agent.name)
    
    user_id = tool_context.session.user_id
    session_id = f"eval_session_{uuid.uuid4().hex}"
    
    sub_session_service = runner.session_service
    await sub_session_service.create_session(
        app_name=eval_agent.name, 
        user_id=user_id, 
        session_id=session_id
    )
    
    query_content = types.Content(role="user", parts=[types.Part(text=eval_prompt)])

    evaluation = ""
    async for event in runner.run_async(
        new_message=query_content,
        user_id=user_id,
        session_id=session_id
    ):
        if event.is_final_response() and event.content:
            evaluation = event.content.parts[0].text
            break
    
    return evaluation

# --- Root Agent (Orchestrator) ---
# This is the main agent that we will interact with. It uses the following concepts:
# 1. LLM-powered Agent
# 2. Custom Tools (the functions above)
# 3. Built-in Tool (preload_memory)
# 4. Session State (via tool_context in the tools)
# 5. Long-Term Memory (via preload_memory)

root_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=RETRY_CONFIG),
    name="StudyAgent",
    
    instruction="""You are an expert, autonomous study helper.
    Your primary goal is to help a user learn any topic.
    You MUST manage the user's progress using your tools and session state.

    Your Capabilities (Tools):
    - `generate_study_plan`: Call this when the user wants to start a new topic.
    - `teach_next_module`: Call this when the user says "start", "next", or "teach me".
    - `generate_quiz`: Call this ONLY when the user asks to be quizzed.
    - `evaluate_answer`: Call this ONLY when the user provides an answer to a quiz.

    How to Behave:
    1.  Analyze the User's Request:
        - If the user's message clearly states a topic they want to learn (e.g., "Teach me python", "I want to learn about Agentic AI"), you MUST call `generate_study_plan` with that topic.
        - If the user's message is a simple greeting (e.g., "Hi", "Teach me") and does NOT include a topic, you MUST ask them what topic they want to learn.
    
    2.  Presenting Plans: After `generate_study_plan` runs, you MUST show the user the returned plan and ask if they are ready to start Module 1.
    
    3.  Teaching & Presenting: When the user is ready to start, you MUST call `teach_next_module`. After the tool runs, you MUST present its full, un-summarized output (the lesson) directly to the user.
    
    4.  Continuing: After *presenting* the lesson, ask them what they want to do next (e.Sg., "next module", "quiz me", or ask a question).
    
    5.  Quizzing & Evaluating:
        - If the user asks for a quiz, call `generate_quiz` and present the quiz questions.
        - If the user provides an answer, call `evaluate_answer` and present the evaluation.
    
    6.  Memory: You have `preload_memory`. This tool automatically loads past user preferences. Acknowledge them. For example, if memory says "User prefers short lessons", you should say "I remember you like short lessons, so I'll keep this concise."

    You are autonomous. Proactively guide the user through their plan.
    """,
    tools=[
        generate_study_plan,
        teach_next_module,
        generate_quiz,
        evaluate_answer,
        preload_memory  # 3B
    ],
)

Overwriting study_agent/agent.py


In [20]:
%%writefile {AGENT_DIR}/__init__.py

# Now, 'study_agent' folder a Python package, and we need to tell the ADK web server to use the LoggingPlugin for observability.

from google.adk.plugins.logging_plugin import LoggingPlugin

# This setup function is called by the ADK web server when it starts. We use it to add the LoggingPlugin (From 4a). This is for logs and tracability.
def setup(app_name: str, **kwargs):
    return {
        "plugins": [LoggingPlugin()]
    }

Overwriting study_agent/__init__.py


In [21]:
# Also set the API key over there
env_content = f"GOOGLE_API_KEY={GOOGLE_API_KEY}\n"

with open(f"{AGENT_DIR}/.env", "w") as f:
    f.write(env_content)

print(f".env file created successfully")

.env file created successfully


# Creating an Evaluation Set

In [22]:
%%writefile {AGENT_DIR}/plan_generation.evalset.json

# This file defines our "golden path" test case (From 4B, Agent Evaluation).
# We can run this from the "Eval" tab in the ADK web UI to ensure our agent's planning logic is working correctly.


{
  "eval_set_id": "plan_generation_test",
  "eval_cases": [
    {
      "eval_id": "agentic_ai_plan",
      "conversation": [
        {
          "user_content": {
            "parts": [
              {
                "text": "I want to learn about 'Agentic AI'"
              }
            ]
          },
          "intermediate_data": {
            "tool_uses": [
              {
                "name": "generate_study_plan",
                "args": {
                  "topic": "Agentic AI"
                }
              }
            ]
          },
          "final_response": {
            "parts": [
              {
                "text": "Study plan created for 'Agentic AI'"
              }
            ]
          }
        }
      ]
    }
  ]
}

Overwriting study_agent/plan_generation.evalset.json


# Run the Agent UI (ADK Agent UI Link)

In [23]:
# This cell runs the helper function from d4a to generate the proxy link for you to access the web UI.

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers
import os
import sys

# ADK looks here
os.environ["AGENT_DIR"] = AGENT_DIR
print(f"AGENT_DIR set to: {os.environ['AGENT_DIR']}")

def get_adk_proxy_url():
    ADK_PORT = 8000
    url_prefix = None
    url = f"http://localhost:{ADK_PORT}"

    # Check if in Google Colab
    if 'google.colab' in sys.modules:
        try:
            from google.colab.output import serve_kernel_port
            url = serve_kernel_port(ADK_PORT, path='/')
            print(f"Colab environment detected. Proxy URL will be: {url}")
            # The adk web command in Colab runs on root, so no prefix is needed
            url_prefix = None
        except ImportError:
            print("Could not import google.colab.output. Defaulting to localhost.")
            url_prefix = None

    # Check if in Kaggle
    elif 'kaggle_secrets' in sys.modules:
        try:
            PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
            servers = list(list_running_servers())
            if not servers:
                raise Exception("No running Jupyter servers found.")

            baseURL = servers[0]["base_url"]
            path_parts = baseURL.split("/")
            kernel = path_parts[2]
            token = path_parts[3]

            url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
            url = f"{PROXY_HOST}{url_prefix}"
            print(f"Kaggle environment detected. Proxy URL: {url}")

        except Exception as e:
            print(f"Kaggle environment check failed ({e}). Defaulting to localhost.")
            url_prefix = None

    # Default for local or other environments
    else:
        print("Assuming local environment. Access at http://localhost:8000")
        url_prefix = None


    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #1a73e8; border-radius: 8px; background-color: #f4f8ff; margin: 20px 0; font-family: sans-serif;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #1a73e8; font-size: 1.2em;">
            <strong>🚀 Your Study Agent is Almost Ready!</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the server.</li>
                <li style="margin-bottom: 5px;">Wait for that cell's output to show it is "Running".</li>
                <li>Once it's running, <strong>click the button below</strong> to open the UI in a new tab.</li>
            </ol>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 12px 24px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: bold;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open Study Agent UI ↗
        </a>
    </div>
    """
    display(HTML(styled_html))
    return url_prefix

# Generate the button and get the URL prefix
url_prefix = get_adk_proxy_url()

AGENT_DIR set to: study_agent
Kaggle environment detected. Proxy URL: https://kkb-production.jupyter-proxy.kaggle.net


# Start the ADK Server

In [ ]:
# 1. Run this cell.
# 2. It will not "complete" but will show "Running...".
# 3. Go back to the cell above and click the "Open Study Agent UI" button.

# To stop the server, just click the "Stop" icon on this cell.

if url_prefix: # For Kaggle Notebooks
    print(f"Starting server with URL prefix: {url_prefix}")
    !adk web --log_level DEBUG --url_prefix {url_prefix}
else: # Colab or Local env
    print("Starting server on port 8000")
    !adk web --log_level DEBUG --port 8000

Starting server with URL prefix: 
/usr/local/lib/python3.11/dist-packages/google/adk/cli/fast_api.py:130: UserWarning: [EXPERIMENTAL] InMemoryCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  credential_service = InMemoryCredentialService()
/usr/local/lib/python3.11/dist-packages/google/adk/auth/credential_service/in_memory_credential_service.py:33: UserWarning: [EXPERIMENTAL] BaseCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__()
INFO:     Started server process [117]
INFO:     Waiting for application startup.

+-----------------------------------------------------------------------------+
| ADK Web Server started                                                      |
|                                                                             |
| 